In [2]:
import cv2
import numpy as np

In [3]:
img = cv2.imread('data/chessboard.png')
pattern_size = (10, 7)

res, corners = cv2.findChessboardCorners(img, pattern_size)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

corners_subpix = cv2.cornerSubPix(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), corners, (10, 10), (-1,-1), criteria)

for c in corners.squeeze():
    img = cv2.circle(img, tuple(c), 10, (0, 0, 255))
cv2.drawChessboardCorners(img, pattern_size, corners_subpix, res)

cv2.imshow('', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)

rms, camera_matrix, dist_coefs, rvecs, tvecs = \
    cv2.calibrateCamera([pattern_points], [corners_subpix], img[0].shape, None, None)

In [9]:
img = cv2.imread('data/chessboard.png')
camera_matrix = np.load('data/camera_mat.npy')
dist_coefs = np.load('data/dist_coefs.npy')

img_undistorted = cv2.undistort(img, camera_matrix, dist_coefs)
cv2.imshow('', img_undistorted)
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
ud_corners = cv2.undistortPoints(corners_subpix, camera_matrix, dist_coefs)
print(ud_corners[0])

[[-0.16817029 -0.11604928]]


In [11]:
ud_corners = np.c_[ud_corners.squeeze(), np.ones(len(ud_corners))]

img_pts, _ = cv2.projectPoints(ud_corners, (0, 0, 0), (0, 0, 0), camera_matrix, None)

for c in img_pts.squeeze().astype(np.float32):
    cv2.circle(img_undistorted, tuple(c), 5, (0, 0, 255), 2)

cv2.imshow('', img_undistorted)
cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
img_pts = ud_corners @ camera_matrix.T
img_pts /= img_pts[:, 2, None]

img_pts = img_pts[:, :2]

for c in img_pts.squeeze().astype(np.float32):
    cv2.circle(img_undistorted, tuple(c), 5, (0, 255, 0), 2)
    
cv2.imshow('', img_undistorted)
cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
ret, rvec, tvec = cv2.solvePnP(pattern_points, corners_subpix, camera_matrix, dist_coefs)